----

# Landsat 7 image reconstruction using autoencoders

**Nota:** El código correspondiente a la arquitectura fue basando en el post de medium de Vipul Vaibhaw. Github: @vaibhawvipul


EL objetivo de este trabajo es tratar de reconstruir una imágen del satélite Landsat luego de que el SLC+ fallase el año 2003, para esto utilizaremos una arquitectura neuronal de tipo autoencoder.

In [179]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

In [180]:
train_root = 'data/train'
val_root = 'data/validation'

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(train_root, transform=transform),
    batch_size = 4, shuffle=True, **kwargs)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(val_root, transform=transform),
    batch_size = 4, shuffle=True, **kwargs)

In [181]:

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(True),
            nn.Conv2d(6,16,kernel_size=5),
            nn.ReLU(True))
        self.decoder = nn.Sequential(             
            nn.ConvTranspose2d(16,6,kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,3,kernel_size=5),
            nn.ReLU(True),
            nn.Sigmoid())
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [185]:
#defining some params
num_epochs = 100 #you can go for more epochs, I am using a mac
batch_size = 4

In [186]:
model = Autoencoder().cpu()
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)

In [188]:
for epoch in range(num_epochs):
    for (data_noise,data_clean) in zip(train_loader,val_loader):
        img_clean, _ = data_clean
        img_clean = Variable(img_clean).cpu()
        img_noise, _ = data_noise
        img_noise = Variable(img_noise).cpu()
        # ===================forward=====================
        output = model(img_noise)
        loss = distance(output, img_clean)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data.numpy()))

epoch [1/100], loss:1.0069
epoch [2/100], loss:0.7540
epoch [3/100], loss:0.8283


KeyboardInterrupt: 

In [170]:
predicts = []
i=0
for data in train_loader:
    print(i)
    i+=1
    img, _ = data
    img = Variable(img).cpu()
    # ===================forward=====================
    output = model(img)
#    print(output)
    predicts.append(output)
#    loss = distance(output, img)

0
1


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not DataLoader